In [1]:
%config IPCompleter.greedy=True

In [2]:
from pprint import pprint
import folium
import geopy.distance

In [5]:
import ee
ee.Authenticate()

Enter verification code:  4/1gEC1dO-NukzTGkawwKf97jYXTVLsBhrUuy8YKZoAbSN4HixLrL5rLM



Successfully saved authorization token.


In [6]:
ee.Initialize()

In [7]:
# constants for each satellite
fine_res_datasets = {
    'ls5': 'LANDSAT/LT05/C01/T1_SR',
    'ls8': 'LANDSAT/LC08/C01/T1_SR',
    's2':  'COPERNICUS/S2_SR'
}
fine_res_resolutions = {
    'ls5': 30,
    'ls8': 30,
    's2':  20
}
fine_res_band_selections = {
    'ls5': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'sr_cloud_qa'],
    'ls8': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa']
}
fine_res_time_field = {
    'ls5': "system:time_start",
    'ls8': "SENSING_TIME"
}

In [8]:
# User inputted data
left_long = 133.84410607205484
right_long = left_long+(134.7422383962736 - left_long)/5
top_lat = -22.638743081339985
bottom_lat = top_lat - (top_lat - (-23.52560283638598))/5
corner_coords = [[right_long, bottom_lat], [left_long, bottom_lat], [left_long, top_lat], [right_long, top_lat], [right_long, bottom_lat]]
date_range = ("2014-08-01", "2015-12-31")
satellite_choice = 'ls8'

In [9]:
# creating a region and defining an imagecollection based on the selected parameters
region = ee.Geometry.Polygon(corner_coords)
collection = ee.ImageCollection(fine_res_datasets[satellite_choice]).filterBounds(region).filterDate(*date_range).sort(fine_res_time_field[satellite_choice])
# pprint(collection.getInfo())
img_list = (collection.map(lambda image: image.clip(region))).toList(collection.size())

In [10]:
# generating an array of the first image
test_img = ee.Image(img_list.get(1))

In [97]:
# creating an array for the 


In [98]:
# Visualizing the image
mapIdDict = (test_img).getMapId({'bands': ['B4', 'B3', 'B2'], 'min': 93, 'max': 1801})

center_x = corner_coords[0][0] + (corner_coords[1][0] - corner_coords[0][0])/2
center_y = corner_coords[1][1] + (corner_coords[1][1] - corner_coords[2][1])/2

map = folium.Map(location=[center_x, center_y])
folium.TileLayer(
    tiles = mapIdDict['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='test_img',
).add_to(map)
map.add_child(folium.LayerControl())
map